In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import torchvision.models as models
import torchsummary
import pandas as pd
import random
from tqdm import tqdm, tqdm_notebook
import numpy as np

In [ ]:
import random

import cv2
from matplotlib import pyplot as plt

import albumentations as A

In [ ]:
import torch
import numpy as np


class Cutout(object):
    """Randomly mask out one or more patches from an image.
    Args:
        n_holes (int): Number of patches to cut out of each image.
        length (int): The length (in pixels) of each square patch.
    """
    def __init__(self, n_holes, length):
        self.n_holes = n_holes
        self.length = length

    def __call__(self, img):
        """
        Args:
            img (Tensor): Tensor image of size (C, H, W).
        Returns:
            Tensor: Image with n_holes of dimension length x length cut out of it.
        """
        h = img.size(1)
        w = img.size(2)

        mask = np.ones((h, w), np.float32)

        for n in range(self.n_holes):
            y = np.random.randint(h)
            x = np.random.randint(w)

            y1 = np.clip(y - self.length // 2, 0, h)
            y2 = np.clip(y + self.length // 2, 0, h)
            x1 = np.clip(x - self.length // 2, 0, w)
            x2 = np.clip(x + self.length // 2, 0, w)

            mask[y1: y2, x1: x2] = 0.

        mask = torch.from_numpy(mask)
        mask = mask.expand_as(img)
        img = img * mask

        return img

In [ ]:
torch.manual_seed(1)
normalize = transforms.Normalize(mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
                                     std=[x / 255.0 for x in [63.0, 62.1, 66.7]])

In [ ]:
train_transform = transforms.Compose([])
train_transform.transforms.append(transforms.RandomCrop(32, padding=4))
train_transform.transforms.append(transforms.RandomHorizontalFlip())
train_transform.transforms.append(transforms.ToTensor())
train_transform.transforms.append(normalize)
train_transform.transforms.append(Cutout(n_holes=1, length=16))

In [ ]:
test_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize])

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    #Cutout(n_holes=1, length=16)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

In [ ]:
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=128, shuffle=False, num_workers=2, pin_memory= True)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, dropRate=0.0):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.relu1 = nn.ReLU(inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes)
        self.relu2 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                               padding=0, bias=False) or None
    def forward(self, x):
        if not self.equalInOut:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)

class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, dropRate=0.0):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate)
    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, dropRate):
        layers = []
        for i in range(nb_layers):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, dropRate))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)

class WideResNet(nn.Module):
    def __init__(self, depth, num_classes, widen_factor=1, dropRate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor]
        assert((depth - 4) % 6 == 0)
        n = (depth - 4) // 6
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, dropRate)
        # 2nd block
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, dropRate)
        # 3rd block
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, dropRate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(nChannels[3])
        self.relu = nn.ReLU(inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)
        self.nChannels = nChannels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))

        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        out = self.fc(out)
        return out

In [ ]:

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.resnet = models.resnet18(pretrained=False, num_classes=10)

        self.resnet.conv1 = torch.nn.Conv2d(
            3, 64, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.resnet.maxpool = torch.nn.Identity()

    def forward(self, x):
        x = self.resnet(x)
        x = F.log_softmax(x, dim=1)

        return x

In [ ]:

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.resnet = torch.hub.load('pytorch/vision:v0.9.0', 'wide_resnet50_2', pretrained=False, num_classes=10)

        self.resnet.conv1 = torch.nn.Conv2d(
            3, 64, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.resnet.maxpool = torch.nn.Identity()

    def forward(self, x):
        x = self.resnet(x)
        x = F.log_softmax(x, dim=1)

        return x

In [ ]:
import pdb; 
def fit_epoch(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    running_corrects = 0
    processed_data = 0
  
    for inputs, labels in train_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)
        model.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        preds = torch.argmax(outputs, 1)

        preds = torch.max(outputs.data, 1)[1]



        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_data += inputs.size(0)
              
    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc

def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0
    loss = 0
    correct = 0
    
    for inputs, labels in val_loader:

        with torch.set_grad_enabled(False):
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(inputs)

            loss += F.nll_loss(outputs, labels, reduction="sum")
            prediction = outputs.max(1)[1]
            correct += prediction.eq(labels.view_as(prediction)).sum().item()

    loss /= len(val_loader.dataset)

    percentage_correct = 100.0 * correct / len(val_loader.dataset)


        
    val_loss = loss
    val_acc = percentage_correct
    return val_loss, val_acc


def train(train_loader, val_loader, model, epochs):
    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    with tqdm(desc="epoch", total=epochs) as pbar_outer:

        #milestones = [25, 40]

        '''
        optimizer = torch.optim.SGD(
            model.parameters(), lr=0.05, momentum=0.9, weight_decay=5e-4
        )
        scheduler = torch.optim.lr_scheduler.MultiStepLR(
            optimizer, milestones=milestones, gamma=0.1
        )
        criterion = F.nll_loss
        '''

        criterion = nn.CrossEntropyLoss().cuda()
        optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, nesterov=True, weight_decay=5e-4)
        scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)



        for epoch in range(epochs):
            train_loss, train_acc = fit_epoch(model, train_loader, criterion, optimizer)
            print("loss", train_loss)            
            val_loss, val_acc = eval_epoch(model, val_loader, criterion)
            history.append((train_loss, train_acc, val_loss, val_acc))            
            pbar_outer.update(1)
            tqdm.write(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                           v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))
            scheduler.step()
            torch.save(model.state_dict(), f'epoch_{epoch+1}' + '.pt')                
    return history


def predict(model, test_loader):
    #pdb.set_trace()
    correct_answer = []
    with torch.no_grad():
        answer = []
    
        for inputs, labels in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            #preds = outputs.max(1)[1].numpy()
            preds = torch.argmax(outputs, 1).numpy()
            answer.extend(preds)
            correct_answer.extend(labels.cpu().numpy())
    return answer, correct_answer

def test(model, loader):
    model.eval()    # Change model to 'eval' mode (BN uses moving mean/var).
    correct = 0.
    total = 0.
    for images, labels in loader:
        images = images.cuda()
        labels = labels.cuda()

        with torch.no_grad():
            pred = model(images)

        pred = torch.max(pred.data, 1)[1]
        total += labels.size(0)
        correct += (pred == labels).sum().item()

    val_acc = correct / total
    model.train()
    return val_acc

def default_trainer(train_loader, test_loader, model, epochs):
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
    train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"
    
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, nesterov=True, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[60, 120, 160], gamma=0.2)
    
    with tqdm(desc="epoch", total=epochs) as pbar_outer:
    
      for epoch in range(200):

        xentropy_loss_avg = 0.
        correct = 0.
        total = 0.
        
        for i, (images, labels) in enumerate(train_loader):
          images = images.cuda()
          labels = labels.cuda()

          model.zero_grad()
          pred = model(images)
          
          xentropy_loss = criterion(pred, labels)
          xentropy_loss.backward()
          optimizer.step()

          xentropy_loss_avg += xentropy_loss.item()

          pred = torch.max(pred.data, 1)[1]
          total += labels.size(0)
          correct += (pred == labels.data).sum().item()
          accuracy = correct / total

        test_acc = test(model, test_loader)
        pbar_outer.update(1)
        tqdm.write(log_template.format(ep=epoch+1, t_loss=xentropy_loss_avg,\
                                           t_acc=accuracy, v_acc=test_acc))
        scheduler.step()
        torch.save(model.state_dict(), f'epoch_{epoch+1}' + '.pt')




In [ ]:
EPOCHS=200
SEED = 1
torch.manual_seed(SEED)


In [ ]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
torch.cuda.is_available()
DEVICE = torch.device("cuda")

CUDA is available!  Training on GPU ...


In [ ]:
clf = WideResNet(depth=28, num_classes=10, widen_factor=10, dropRate=0.3) 


In [ ]:
clf.load_state_dict(torch.load('epoch_127.pt'))

<All keys matched successfully>

In [ ]:
#clf = WideResNet(depth=28, num_classes=10, widen_factor=10, dropRate=0.3) 
clf = clf.to('cuda')
default_trainer(trainloader, testloader, model=clf, epochs=EPOCHS)

epoch:   0%|          | 1/200 [04:12<13:56:07, 252.10s/it]


Epoch 001 train_loss: 238.1638     train_acc 0.7912 val_acc 0.7977


epoch:   1%|          | 2/200 [08:28<13:55:41, 253.24s/it]


Epoch 002 train_loss: 189.8071     train_acc 0.8331 val_acc 0.8401


epoch:   2%|▏         | 3/200 [12:43<13:53:26, 253.84s/it]


Epoch 003 train_loss: 178.7974     train_acc 0.8430 val_acc 0.8483


epoch:   2%|▏         | 4/200 [16:59<13:51:06, 254.42s/it]


Epoch 004 train_loss: 173.9015     train_acc 0.8463 val_acc 0.8701


epoch:   2%|▏         | 4/200 [20:25<16:40:49, 306.37s/it]


KeyboardInterrupt: ignored

In [ ]:
clf.load_state_dict(torch.load('epoch_123.pt'))

<All keys matched successfully>

In [ ]:
clf = clf.to('cuda')


In [ ]:
  
clf = clf.to('cuda')
history = train(trainloader, testloader, model=clf, epochs=EPOCHS)
loss, acc, val_loss, val_acc = zip(*history)
plt.figure(figsize=(15, 9))
plt.plot(loss, label="train_loss")
plt.plot(val_loss, label="val_loss")
plt.legend(loc='best')
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()
plt.close()


epoch:   0%|          | 0/200 [00:00<?, ?it/s]

loss 0.5633172700309753


epoch:   0%|          | 1/200 [04:24<14:36:47, 264.36s/it]


Epoch 001 train_loss: 0.5633     val_loss -8.1512 train_acc 0.8055 val_acc 81.4800
loss 0.5567567379379272


epoch:   1%|          | 2/200 [08:54<14:38:08, 266.10s/it]

...


Epoch 043 train_loss: 0.4361     val_loss -9.3405 train_acc 0.8503 val_acc 86.1600
loss 0.437901500492096


epoch:  22%|██▏       | 44/200 [3:17:09<11:38:21, 268.60s/it]


Epoch 044 train_loss: 0.4379     val_loss -9.2886 train_acc 0.8485 val_acc 87.2000
loss 0.43831404205322266


epoch:  22%|██▎       | 45/200 [3:21:37<11:33:51, 268.59s/it]


Epoch 045 train_loss: 0.4383     val_loss -9.1358 train_acc 0.8489 val_acc 84.2400


In [ ]:
answ, correct = predict(clf, testloader)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(correct, answ))

0.963


In [ ]:
submit = pd.DataFrame(columns=['index'])
submit['index'] = np.arange(len(testset))

In [ ]:
submit['label'] = answ

In [ ]:
submit.head

<bound method NDFrame.head of       index  label
0         0      3
1         1      8
2         2      8
3         3      0
4         4      6
...     ...    ...
9995   9995      8
9996   9996      3
9997   9997      5
9998   9998      1
9999   9999      7

[10000 rows x 2 columns]>

In [ ]:
submit[['index', 'label']].to_csv('new_answer.csv', index=False)